In [17]:
from enum import Enum
import math

In [40]:
filepath = 'data/day15.txt'

In [113]:
class EntityType:
    ELF = 'E'
    GOBLIN = 'G'
    
class Direction(Enum):
    UP = 0
    DOWN = 1
    LEFT = 2
    RIGHT = 3

class Entity:
    def __init__(self, x, y, hp, enemy):
        self.x = x
        self.y = y
        self.hp = hp
        self.enemy = enemy
    
    @property
    def type(self):
        pass
    
    def update(self, world):
        pass
    
    def get_enemies(self, world):
        width, height = len(world.world[0]), len(world.world)
        best_distance = math.inf
        searched_spots = {}
        # do flood fill
        def flood_fill(x, y, dist=0):
            # if distance is greater than best distance ignore
            if dist > best_distance:
                return False
            # if already searched and distance shorter ignore
            prev_dist = searched_spots.setdefault(x, {}).setdefault(y, math.inf)
            if prev_dist <= dist:
                return False
            # otherwise update with shorter path
            print('Checking {},{},{}'.format(x, y, dist))
            searched_spots[x][y] = dist
            # if outside
            if y < 0 or y >= height or\
               x < 0 or x >= width:
                return False
            cell = world.world[y][x]
            # if wall ignore
            if cell == World.WALL:
                return False
            # check enemies
            if x in world.entity_cache and y in world.entity_cache[x]:
                entity = world.entity_cache[x][y]
                if entity is not self and entity.type == self.enemy:
                    return True
            # if nothing search around
            results = []
            results.append((Direction.UP,   flood_fill(x, y-1, dist+1))) # top 
            results.append((Direction.DOWN, flood_fill(x, y+1, dist+1))) # bot
            results.append((Direction.LEFT, flood_fill(x-1, y, dist+1))) # left
            results.append((Direction.RIGHT,flood_fill(x+1, y, dist+1))) # right
            # search results
            for direction, result in results:
                # if enemy found, result
                if result and (dist+1) < best_distance:
                    best_distance = dist+1
                    return result
            return False
        # perform flood fill
        result = flood_fill(self.x, self.y)
        return result
    
    def __str__(self):
        return '{} @{},{}'.format(self.type, self.x, self.y)
    
    def __repr__(self):
        return self.__str__()

In [114]:
class Goblin(Entity):
    def __init__(self, x, y, hp):
        super().__init__(x, y, hp, EntityType.ELF)
    
    @property
    def type(self):
        return EntityType.GOBLIN
    
    def update(self, world):
        return

In [103]:
class Elf(Entity):
    def __init__(self, x, y, hp):
        super().__init__(x, y, hp, EntityType.GOBLIN)
        
    @property
    def type(self):
        return EntityType.ELF
    
    def update(self, world):
        return

In [115]:
class World:
    WALL = '#'
    EMPTY = '.'
    
    def __init__(self):
        self.world = [] # 2d grid environment
        self.height = 0
        self.width = 0
        self.entities = []
        self.entity_cache = {}
    
    def add_world_row(self, row):
        row = row.replace('\n', '')\
                 .replace('E', World.EMPTY)\
                 .replace('G', World.EMPTY)
        row = list(row)
        if not self.width:
            self.width = len(row)
        elif self.width != len(row):
            raise ValueError('Prev width {}, Got width {} from line {}'\
                                .format(self.width, len(row), row))
        self.world.append(row)
        self.height += 1
    
    def add_entity(self, entity):
        x, y = entity.x, entity.y
        cell = self.world[y][x]
        if cell is World.WALL:
            raise ValueError('Cell[{}][{}] is occupied by {}'.format(x, y, cell))
        self.entities.append(entity)
        self.entity_cache.setdefault('x', {})[y] = entity

    def update(self):
        self.entities.sort(key=self.get_global_index)
        self.dead_entities = []
        for entity in self.entities:
            if entity in self.dead_entities:
                continue
            # entity returns the entity that died in that tick
            # could be another entity, or itself
            killed_entity = entity.update(world)
            if killed_entity is not None:
                self.dead_entities.append(killed_entity)
        for entity in dead_entities:
            self.entities.remove(entity)
        self.update_entity_cache()
    
    def update_entity_cache(self):
        self.entity_cache = {}
        for entity in entities:
            self.entity_cache.setdefault(entity.x, {})
            if entity.y in self.entity_cache[entity.x]:
                raise ValueError('Entity position conflict @{},{}'\
                                 .format(entity.x, entity.y))
            self.entity_cache[entity.x][entity.y] = entity
    
    def get_global_index(self, entity):
        return entity.x + entity.y*self.width
    
    def __str__(self):
        buffer = []
        for row in self.world:
            buffer.append(list(row))
        for entity in self.entities:
            buffer[entity.y][entity.x] = entity.type
        return '\n'.join([''.join(row) for row in buffer])

In [116]:
class WorldParser:
    def __init__(self, default_hp=200):
        self.elf_hp = default_hp
        self.goblin_hp = default_hp
    
    def read_from_file(self, filepath):
        with open(filepath, 'r') as file:
            world = self.parse_file(file)
        return world
    
    def parse_file(self, file_object):
        world = World()
        print(world)
        for y, line in enumerate(file_object.readlines()):
            world.add_world_row(line)
            for x, char in enumerate(line):
                if char == EntityType.ELF:
                    world.add_entity(Elf(x, y, self.elf_hp))
                elif char == EntityType.GOBLIN:
                    world.add_entity(Goblin(x, y, self.goblin_hp))
        return world

In [117]:
worldParser = WorldParser()
world = worldParser.read_from_file(filepath)

In [118]:
world.entities[0]

G @14,6

In [119]:
world.entities[0].get_enemies(world)

Checking 14,6,0
Checking 14,5,1
Checking 14,7,1
Checking 14,8,2
Checking 14,9,3
Checking 14,10,4
Checking 14,11,5
Checking 14,12,6
Checking 13,11,6
Checking 13,10,7
Checking 13,9,8
Checking 13,8,9
Checking 13,7,10
Checking 13,6,11
Checking 12,7,11
Checking 12,6,12
Checking 12,5,13
Checking 11,6,13
Checking 11,5,14
Checking 11,7,14
Checking 11,8,15
Checking 11,9,16
Checking 11,10,17
Checking 10,9,17
Checking 10,8,18
Checking 10,7,19
Checking 10,6,20
Checking 9,7,20
Checking 9,6,21
Checking 9,8,21
Checking 9,9,22
Checking 9,10,23
Checking 8,9,23
Checking 8,8,24
Checking 8,10,24
Checking 7,9,24
Checking 7,8,25
Checking 7,10,25
Checking 7,11,26
Checking 7,12,27
Checking 7,13,28
Checking 7,14,29
Checking 7,15,30
Checking 7,16,31
Checking 7,17,32
Checking 7,18,33
Checking 7,19,34
Checking 7,20,35
Checking 6,19,35
Checking 6,18,36
Checking 6,17,37
Checking 6,16,38
Checking 6,15,39
Checking 6,14,40
Checking 6,13,41
Checking 6,12,42
Checking 6,11,43
Checking 6,10,44
Checking 5,11,44
Checking 5,

Checking 22,24,154
Checking 23,25,154
Checking 23,24,155
Checking 24,25,155
Checking 24,24,156
Checking 24,23,157
Checking 25,24,157
Checking 25,25,156
Checking 20,27,151
Checking 21,27,152
Checking 21,28,153
Checking 22,27,153
Checking 21,26,151
Checking 22,26,152
Checking 23,26,153
Checking 23,27,154
Checking 24,26,154
Checking 24,27,155
Checking 24,28,156
Checking 25,27,156
Checking 25,26,155
Checking 26,26,156
Checking 26,25,157
Checking 26,27,157
Checking 27,26,157
Checking 18,25,147
Checking 18,24,146
Checking 16,23,143
Checking 16,22,144
Checking 17,23,144
Checking 16,22,142
Checking 16,21,141
Checking 16,20,142
Checking 17,21,142
Checking 17,20,143
Checking 17,22,143
Checking 18,21,143
Checking 18,20,144
Checking 18,22,144
Checking 19,21,144
Checking 19,20,145
Checking 19,19,146
Checking 20,20,146
Checking 20,19,147
Checking 20,18,148
Checking 21,19,148
Checking 21,18,149
Checking 21,17,150
Checking 21,16,151
Checking 21,15,152
Checking 21,14,153
Checking 21,13,154
Checking 21,

Checking 25,21,190
Checking 25,22,191
Checking 26,21,191
Checking 26,22,192
Checking 27,21,192
Checking 22,20,188
Checking 21,20,189
Checking 24,20,188
Checking 25,20,189
Checking 26,20,190
Checking 27,20,191
Checking 22,19,187
Checking 22,18,186
Checking 27,18,183
Checking 25,17,182
Checking 25,16,183
Checking 24,16,184
Checking 23,16,185
Checking 22,16,186
Checking 24,17,183
Checking 23,17,184
Checking 22,17,185
Checking 27,17,182
Checking 25,16,181
Checking 25,15,182
Checking 24,15,183
Checking 23,15,184
Checking 22,15,185
Checking 24,16,182
Checking 23,16,183
Checking 22,16,184
Checking 27,16,181
Checking 28,16,182
Checking 25,15,180
Checking 25,14,181
Checking 24,14,182
Checking 24,13,183
Checking 24,12,184
Checking 23,13,184
Checking 23,12,185
Checking 22,13,185
Checking 22,12,186
Checking 23,14,183
Checking 22,14,184
Checking 24,15,181
Checking 23,15,182
Checking 22,15,183
Checking 27,15,180
Checking 28,15,181
Checking 29,15,182
Checking 25,14,179
Checking 25,13,180
Checking 24,

Checking 28,15,167
Checking 29,15,168
Checking 24,16,162
Checking 25,16,163
Checking 26,16,164
Checking 27,16,165
Checking 28,16,166
Checking 24,17,161
Checking 25,17,162
Checking 26,17,163
Checking 27,17,164
Checking 23,16,159
Checking 23,15,160
Checking 23,14,161
Checking 23,13,162
Checking 23,12,163
Checking 24,13,163
Checking 24,12,164
Checking 25,13,164
Checking 24,14,162
Checking 25,14,163
Checking 26,14,164
Checking 26,13,165
Checking 27,13,166
Checking 28,13,167
Checking 29,13,168
Checking 30,13,169
Checking 27,14,165
Checking 28,14,166
Checking 29,14,167
Checking 30,14,168
Checking 24,15,161
Checking 25,15,162
Checking 26,15,163
Checking 27,15,164
Checking 28,15,165
Checking 29,15,166
Checking 24,16,160
Checking 25,16,161
Checking 26,16,162
Checking 27,16,163
Checking 28,16,164
Checking 23,15,158
Checking 23,14,159
Checking 23,13,160
Checking 23,12,161
Checking 24,13,161
Checking 24,12,162
Checking 25,13,162
Checking 24,14,160
Checking 25,14,161
Checking 26,14,162
Checking 26,

Checking 30,4,168
Checking 28,5,165
Checking 27,6,163
Checking 28,6,164
Checking 26,7,161
Checking 27,7,162
Checking 28,7,163
Checking 29,7,164
Checking 26,8,160
Checking 27,8,161
Checking 28,8,162
Checking 29,8,163
Checking 30,8,164
Checking 30,7,165
Checking 30,9,165
Checking 31,8,165
Checking 24,9,157
Checking 25,9,158
Checking 25,10,159
Checking 26,9,159
Checking 27,9,160
Checking 28,9,161
Checking 29,9,162
Checking 20,10,154
Checking 19,10,155
Checking 18,10,156
Checking 22,10,154
Checking 23,10,155
Checking 23,11,156
Checking 24,10,156
Checking 20,11,153
Checking 19,11,154
Checking 18,11,155
Checking 17,11,156
Checking 17,10,157
Checking 17,12,157
Checking 16,11,157
Checking 16,10,158
Checking 16,12,158
Checking 15,11,158
Checking 15,10,159
Checking 15,12,159
Checking 22,11,153
Checking 20,12,152
Checking 19,12,153
Checking 18,12,154
Checking 22,12,152
Checking 20,13,151
Checking 19,13,152
Checking 22,13,151
Checking 23,13,152
Checking 23,12,153
Checking 24,13,153
Checking 24,12,

...

Checking 8,21,27
Checking 7,21,28
Checking 6,21,29
Checking 5,21,30
Checking 4,21,31
Checking 3,21,32
Checking 2,21,33
Checking 1,21,34
Checking 0,21,35
Checking 12,21,25
Checking 13,21,26
Checking 14,21,27
Checking 15,21,28
Checking 15,20,29
Checking 16,21,29
Checking 16,20,30
Checking 17,21,30
Checking 17,20,31
Checking 17,22,31
Checking 18,21,31
Checking 18,20,32
Checking 18,22,32
Checking 19,21,32
Checking 19,20,33
Checking 19,19,34
Checking 20,20,34
Checking 20,19,35
Checking 20,18,36
Checking 21,19,36
Checking 21,18,37
Checking 21,17,38
Checking 21,16,39
Checking 21,15,40
Checking 21,14,41
Checking 21,13,42
Checking 21,12,43
Checking 21,11,44
Checking 21,10,45
Checking 21,9,46
Checking 21,8,47
Checking 20,9,47
Checking 20,8,48
Checking 20,7,49
Checking 19,8,49
Checking 19,7,50
Checking 18,8,50
Checking 18,7,51
Checking 17,8,51
Checking 17,7,52
Checking 17,6,53
Checking 16,7,53
Checking 16,6,54
Checking 15,7,54
Checking 15,6,55
Checking 15,5,56
Checking 16,8,52
Checking 15,8,53
Ch

Checking 7,25,28
Checking 7,26,29
Checking 7,27,30
Checking 7,28,31
Checking 6,27,31
Checking 6,28,32
Checking 5,27,32
Checking 5,28,33
Checking 4,27,33
Checking 4,28,34
Checking 4,29,35
Checking 4,30,36
Checking 4,31,37
Checking 3,30,37
Checking 3,31,38
Checking 2,30,38
Checking 5,30,37
Checking 3,29,36
Checking 2,29,37
Checking 5,29,36
Checking 3,28,35
Checking 2,28,36
Checking 1,28,37
Checking 3,27,34
Checking 2,27,35
Checking 1,27,36
Checking 8,27,31
Checking 6,26,30
Checking 5,26,31
Checking 4,26,32
Checking 3,26,33
Checking 2,26,34
Checking 1,26,35
Checking 6,25,29
Checking 5,25,30
Checking 4,25,31
Checking 3,25,32
Checking 2,25,33
Checking 1,25,34
Checking 0,25,35
Checking 12,25,25
Checking 13,25,26
Checking 14,25,27
Checking 10,24,24
Checking 9,24,25
Checking 8,24,26
Checking 7,24,27
Checking 6,24,28
Checking 5,24,29
Checking 4,24,30
Checking 3,24,31
Checking 2,24,32
Checking 1,24,33
Checking 0,24,34
Checking 12,24,24
Checking 13,24,25
Checking 14,24,26
Checking 15,24,27
Checki

Checking 0,17,25
Checking 12,17,15
Checking 10,16,14
Checking 9,16,15
Checking 8,16,16
Checking 7,16,17
Checking 6,16,18
Checking 5,16,19
Checking 4,16,20
Checking 3,16,21
Checking 2,16,22
Checking 1,16,23
Checking 0,16,24
Checking 12,16,14
Checking 10,15,13
Checking 9,15,14
Checking 8,15,15
Checking 7,15,16
Checking 6,15,17
Checking 5,15,18
Checking 4,15,19
Checking 12,15,13
Checking 10,14,12
Checking 10,13,11
Checking 9,13,12
Checking 9,14,13
Checking 8,14,14
Checking 7,14,15
Checking 6,14,16
Checking 5,14,17
Checking 4,14,18
Checking 3,14,19
Checking 3,15,20
Checking 2,14,20
Checking 2,13,21
Checking 2,15,21
Checking 1,14,21
Checking 1,13,22
Checking 1,15,22
Checking 0,15,23
Checking 0,14,22
Checking 8,13,13
Checking 8,12,14
Checking 8,11,15
Checking 7,12,15
Checking 6,12,16
Checking 5,12,17
Checking 4,12,18
Checking 7,13,14
Checking 6,13,15
Checking 5,13,16
Checking 4,13,17
Checking 3,13,18
Checking 11,12,9
Checking 10,12,10
Checking 9,12,11
Checking 12,11,7
Checking 11,11,8
Checki

Checking 27,12,27
Checking 28,12,28
Checking 29,12,29
Checking 27,11,26
Checking 28,11,27
Checking 29,11,28
Checking 26,10,24
Checking 27,10,25
Checking 28,10,26
Checking 26,9,23
Checking 27,9,24
Checking 28,9,25
Checking 29,9,26
Checking 22,10,18
Checking 23,10,19
Checking 23,11,20
Checking 24,10,20
Checking 21,9,16
Checking 21,8,17
Checking 22,9,17
Checking 22,8,18
Checking 23,9,18
Checking 23,8,19
Checking 23,7,20
Checking 24,8,20
Checking 24,7,21
Checking 25,8,21
Checking 25,7,22
Checking 25,6,23
Checking 25,5,24
Checking 24,6,24
Checking 26,6,24
Checking 26,5,25
Checking 26,4,26
Checking 27,5,26
Checking 27,4,27
Checking 27,3,28
Checking 28,4,28
Checking 28,3,29
Checking 29,4,29
Checking 29,3,30
Checking 29,2,31
Checking 29,1,32
Checking 29,0,33
Checking 28,1,33
Checking 30,1,33
Checking 30,0,34
Checking 31,1,34
Checking 28,2,32
Checking 30,2,32
Checking 31,2,33
Checking 30,3,31
Checking 31,3,32
Checking 29,5,30
Checking 30,4,30
Checking 28,5,27
Checking 27,6,25
Checking 28,6,26
C

False